In [1]:
import pandas as pd
from spacy.cli import download
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import spacy

In [3]:
train_data = pd.read_csv('../train_data.csv')

In [4]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text
0,https://api.github.com/repos/kubernetes/kubern...,275859420,Kubelet flag precedence order vs files/ConfigM...,See https://docs.google.com/document/d/18-MsCh...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubelet', 'area/kubelet-api']",56097,"['cmd/kubelet/kubelet.go', 'hack/make-rules/te...","['modified', 'modified', 'modified', 'modified...","[7, 1, 21, 1, 1, 1, 1, 1, 1, 15, 139, 146]","[1, 0, 4, 0, 0, 1, 1, 1, 1, 0, 9, 0]","[8, 1, 25, 1, 1, 2, 2, 2, 2, 15, 148, 146]",NaN,kubelet flag precedence order vs dangerous see...
1,https://api.github.com/repos/kubernetes/kubead...,262492428,Individual control of preflight checks,Many times users know better than kubeadm arou...,https://github.com/kubernetes/kubeadm/issues/480,https://github.com/kubernetes/kubernetes/pull/...,['area/kubeadm'],56072,['cmd/kubeadm/app/apis/kubeadm/validation/BUIL...,"['modified', 'modified', 'modified', 'modified...","[1, 26, 29, 20, 21, 1, 3, 17, 4, 2, 6, 3, 5, 6...","[0, 1, 0, 14, 15, 0, 2, 9, 0, 0, 1, 7, 1, 0, 0...","[1, 27, 29, 34, 36, 1, 5, 26, 4, 2, 7, 10, 6, ...",New example in 1.8.0:\r\nsystemctl start kubel...,individual control preflight checks many times...
2,https://api.github.com/repos/kubernetes/kubern...,275470204,seccomp is an alpha feature and not feature gated,see #55983,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubelet', 'area/kubelet-api']",55983,"['cmd/kubelet/app/options/options.go', 'cmd/ku...","['modified', 'modified', 'modified', 'modified...","[5, 6, 0, 0, 0, 0, 0, 0, 5]","[1, 3, 1, 1, 2, 4, 2, 2, 3]","[6, 9, 1, 1, 2, 4, 2, 2, 8]",NaN,seccomp an alpha feature not feature gated see
3,https://api.github.com/repos/kubernetes/kubern...,274962003,kubeletconfig API group is missing a fuzzer ro...,Solution: I will add one.,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet-api'],55961,"['pkg/kubelet/apis/kubeletconfig/BUILD', 'pkg/...","['modified', 'added', 'added', 'modified', 'ad...","[1, 32, 100, 12, 32, 6]","[0, 0, 0, 1, 0, 6]","[1, 32, 100, 13, 32, 12]",/cc @luxas xref #55961 Thanks for sending a P...,kubeletconfig api group missing fuzzer round t...
4,https://api.github.com/repos/kubernetes/kubern...,388343406,api timeouts when using token authentication,<!-- Please use this template while reporting ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/apiserver'],71816,"['cmd/kube-apiserver/app/server.go', 'pkg/cont...","['modified', 'modified', 'modified', 'modified...","[6, 16, 2, 35, 2, 34, 10]","[1, 3, 0, 1, 0, 1, 3]","[7, 19, 2, 36, 2, 35, 13]",@kubernetes/sig-auth-bugs @jmcmeek: Reiteratin...,api timeouts when using token authentication p...


In [5]:
# check for null values
train_data.isnull().sum()


comments_url     0
id               0
title            0
body             3
issue_url        0
pr_url           0
labels           0
pr_number        0
filename         0
status           0
additions        0
deletions        0
changes          0
all_comments    24
all_text         3
dtype: int64

In [7]:
# remove all_text that have null values
train_data = train_data[train_data['all_text'].notnull()]

# remove all_text that has value 'NaN ' or 'nan'
train_data = train_data[~train_data['all_text'].isin(['NaN ', 'nan'])]

Check the label distribution

In [8]:
# Flatten the list of labels if it's a list of lists
flattened_labels = []
for label_list in train_data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    flattened_labels.extend(clean_label_list)

# Print labels distribution count
label_counts = Counter(flattened_labels)
print("Label Distribution Count (Training Data):")

for i, (label, count) in enumerate(label_counts.most_common()):
    print(f"{i+1}. {label}: {count}")

Label Distribution Count (Training Data):
1. area/test: 328
2. area/apiserver: 115
3. area/kubectl: 109
4. area/kubelet: 100
5. area/kubeadm: 74
6. area/e2e-test-framework: 69
7. area/cloudprovider: 50
8. area/dependency: 47
9. area/code-generation: 39
10. area/conformance: 19
11. area/release-eng: 17
12. area/provider/azure: 14
13. area/provider/gcp: 11
14. area/ipvs: 9
15. area/kube-proxy: 6
16. area/kubelet-api: 5
17. area/custom-resources: 5
18. area/admission-control: 5


In [32]:
# Flatten the list of labels if it's a list of lists
flattened_labels = []
for label_list in val_data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    flattened_labels.extend(clean_label_list)

# Print labels distribution count
label_counts = Counter(flattened_labels)
print("Label Distribution Count (Validation Data):")

for i, (label, count) in enumerate(label_counts.most_common()):
    print(f"{i+1}. {label}: {count}")

Label Distribution Count (Validation Data):
1. area/test: 100
2. area/kubelet: 57
3. area/apiserver: 37
4. area/kubectl: 23
5. area/code-generation: 22
6. area/e2e-test-framework: 17
7. area/kubeadm: 11
8. area/kube-proxy: 7
9. area/release-eng: 3
10. area/provider/gcp: 3
11. area/conformance: 2
12. area/cloudprovider: 2
13. area/dependency: 2


Stopwords Removal

In [10]:
# remove stopwords using spaCy
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    download('en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')

train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.text for word in nlp(x) if not word.is_stop]))

In [11]:
# check null values in all_text column
# Display null values for all columns
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


Data Lemmatization

In [12]:
# lematize the text using spaCy
train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.lemma_ for word in nlp(x)]))

In [13]:
# check all_text column for null values
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


Add filename to the text

In [14]:
# add filename to the data
# Only append filename if it exists (not null/None)
train_data['all_text'] = train_data.apply(lambda row: row['all_text'] + ' ' + row['filename'] if pd.notna(row['filename']) else row['all_text'], axis=1)

In [15]:
# check all_text column for null values
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


# Export the Data

In [16]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text
0,https://api.github.com/repos/kubernetes/kubern...,275859420,Kubelet flag precedence order vs files/ConfigM...,See https://docs.google.com/document/d/18-MsCh...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubelet', 'area/kubelet-api']",56097,"['cmd/kubelet/kubelet.go', 'hack/make-rules/te...","['modified', 'modified', 'modified', 'modified...","[7, 1, 21, 1, 1, 1, 1, 1, 1, 15, 139, 146]","[1, 0, 4, 0, 0, 1, 1, 1, 1, 0, 9, 0]","[8, 1, 25, 1, 1, 2, 2, 2, 2, 15, 148, 146]",NaN,kubelet flag precedence order vs dangerous doc...
1,https://api.github.com/repos/kubernetes/kubead...,262492428,Individual control of preflight checks,Many times users know better than kubeadm arou...,https://github.com/kubernetes/kubeadm/issues/480,https://github.com/kubernetes/kubernetes/pull/...,['area/kubeadm'],56072,['cmd/kubeadm/app/apis/kubeadm/validation/BUIL...,"['modified', 'modified', 'modified', 'modified...","[1, 26, 29, 20, 21, 1, 3, 17, 4, 2, 6, 3, 5, 6...","[0, 1, 0, 14, 15, 0, 2, 9, 0, 0, 1, 7, 1, 0, 0...","[1, 27, 29, 34, 36, 1, 5, 26, 4, 2, 7, 10, 6, ...",New example in 1.8.0:\r\nsystemctl start kubel...,individual control preflight check time user k...
2,https://api.github.com/repos/kubernetes/kubern...,275470204,seccomp is an alpha feature and not feature gated,see #55983,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubelet', 'area/kubelet-api']",55983,"['cmd/kubelet/app/options/options.go', 'cmd/ku...","['modified', 'modified', 'modified', 'modified...","[5, 6, 0, 0, 0, 0, 0, 0, 5]","[1, 3, 1, 1, 2, 4, 2, 2, 3]","[6, 9, 1, 1, 2, 4, 2, 2, 8]",NaN,seccomp alpha feature feature gate ['cmd/kubel...
3,https://api.github.com/repos/kubernetes/kubern...,274962003,kubeletconfig API group is missing a fuzzer ro...,Solution: I will add one.,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet-api'],55961,"['pkg/kubelet/apis/kubeletconfig/BUILD', 'pkg/...","['modified', 'added', 'added', 'modified', 'ad...","[1, 32, 100, 12, 32, 6]","[0, 0, 0, 1, 0, 6]","[1, 32, 100, 13, 32, 12]",/cc @luxas xref #55961 Thanks for sending a P...,kubeletconfig api group miss fuzzer round trip...
4,https://api.github.com/repos/kubernetes/kubern...,388343406,api timeouts when using token authentication,<!-- Please use this template while reporting ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/apiserver'],71816,"['cmd/kube-apiserver/app/server.go', 'pkg/cont...","['modified', 'modified', 'modified', 'modified...","[6, 16, 2, 35, 2, 34, 10]","[1, 3, 0, 1, 0, 1, 3]","[7, 19, 2, 36, 2, 35, 13]",@kubernetes/sig-auth-bugs @jmcmeek: Reiteratin...,api timeout token authentication use template ...


In [17]:
# count nan values in the train_data
train_data.isna().sum()

comments_url     0
id               0
title            0
body             0
issue_url        0
pr_url           0
labels           0
pr_number        0
filename         0
status           0
additions        0
deletions        0
changes          0
all_comments    23
all_text         0
dtype: int64

In [ ]:
# export the train_data and val_data to csv
train_data.to_csv('../../../Data/Preprocessed Data/kind:feature/Augmented Data/Preprocessed Data/Original Data/train_data.csv', index=False)